In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver

# Fonction pour extraire le texte d'un élément HTML
def extract_text(element, class_name=None, style=None, text_contains=None):
    if element:
        tag = element.find(class_=class_name, style=style, text=text_contains)
        return tag.text.strip() if tag else ""
    else:
        return ""

# Fonction pour nettoyer le texte
def clean_text(text):
    if text is not None:
        cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
        return cleaned_text.strip() if cleaned_text else None
    else:
        return None

# Fonction pour extraire la date à partir d'un élément HTML
def extract_date(element, text_contains):
    date_elements = element.find_all('a', class_='text')
    date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
    return date

# Fonction pour extraire les détails des offres d'emploi
def scrap_job_details(urls):
    # En-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # Liste pour stocker les détails de chaque emploi
    all_job_details = []

    # Parcourir les liens
    for url in urls:
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_='box row')

        # Parcourir les offres d'emploi sur la page principale
        for offre in offres:
            # Trouver la balise <h4> dans la structure HTML pour extraire le lien
            offre_link_tag = offre.find('h4')

            # Vérifier si la balise <h4> a été trouvée
            if offre_link_tag:
                # Extraire le lien de l'attribut 'href'
                offre_link = offre_link_tag.find('a')['href']
                all_job_details.append({'Offre_Link': offre_link})

    # Fermer le pilote Selenium à la fin
    driver.quit()

    # Concaténer tous les détails des emplois en un seul DataFrame
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
        return all_job_details_df
    else:
        print("Aucun détail d'offre d'emploi trouvé.")
        return None

# Fonction pour récupérer les données des offres d'emploi
def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=200)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='box row')

    data_list = []

    for wrapper in job_description_wrappers:
        h4_tag = wrapper.find('h4')
        poste = extract_text(h4_tag)

        entry_title_tag = wrapper.find('p', class_='entry-title')
        sous_titre = extract_text(entry_title_tag)

        a_text_tag = wrapper.find('a', class_='text')
        code = extract_text(a_text_tag, style='color:#FF0000;')

        date_edition = extract_date(wrapper, "Date d'édition:")
        date_limite = extract_date(wrapper, "Date limite:")

        pays_tag = wrapper.find('a', class_='text')
        pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

        sous_titre = clean_text(sous_titre)

        data_list.append({
            'Poste': clean_text(poste),
            'Sous_titre': sous_titre,
            'Code': clean_text(code),
            'Date_DEdition': date_edition,
            'Date_limite': date_limite,
            'Pays': clean_text(pays),
            'URL': url  # Ajout de la colonne 'URL'
        })

    df = pd.DataFrame(data_list)
    return df

# Fonction pour ajouter la colonne 'Offre_Link' à un DataFrame existant
def add_offre_link_column(result_df):
    job_details_df = scrap_job_details(list(result_df["URL"]))
    if job_details_df is not None:
        # Join des DataFrames
        result_df = pd.concat([result_df, job_details_df], axis=1)
        return result_df
    else:
        print("Impossible d'ajouter la colonne 'Offre_Link' au DataFrame.")
        return None

# Liste des liens
urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

# Créer un DataFrame à partir des liens
result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Supprimer les lignes dont toutes les variables n'ont pas de données
#result_df = result_df.dropna(how='any').reset_index(drop=True)

# Ajouter la colonne 'Offre_Link'
result_df = add_offre_link_column(result_df)
# Supprimer les lignes avec plus de 80% de valeurs NaN
threshold = int(result_df.shape[1] * 0.8)  # 80% des colonnes
result_df = result_df.dropna(thresh=threshold)
result_df=result_df.reset_index(drop=True)
# Afficher le DataFrame
result_df

,Poste,Sous_titre,Code,Date_DEdition,Date_limite,Pays,URL,Offre_Link
1,DECLARANT EN DOUANE,EXCELIAMrecruteDECLARANT EN DOUANEDescription ...,109945,14/02/2024,14/03/2024,d'I...,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114121-ing...
2,INGENIEUR MODELISATION RESEAU,AFRIJOBrecruteINGENIEUR MODELISATION RESEAUDes...,109944,14/02/2024,26/02/2024,d'I...,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114120-age...
3,AGENT MARKETING ET COMMUNICATION,WAGSystemsrecruteAGENT MARKETING ET COMMUNICAT...,109943,14/02/2024,26/02/2024,d'I...,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114119-vis...
4,VISA SPECIALIST,HUAWEI TECHNOLOGIES COTE D'IVOIRE SAUrecruitVI...,109942,14/02/2024,26/02/2024,d'I...,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114118-pli...
5,PLIEUR,AGENCE IVOIRE INTERIM (A2I)recrutePLIEURDescri...,109941,14/02/2024,29/02/2024,FRANCE,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114117-com...
...,...,...,...,...,...,...,...,...
619,STAGE DE SOUTENANCE (RH-COM; GESCOM; FCGE; RIT...,Description du posteStage de soutenanceNous of...,95492,14/02/2023,27/03/2024,...,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114083-jun...
620,ESTHÉTICIENNE - PROTHÉSISTE ONGULAIRE (H/F),Description du posteNous recherchons des esthé...,94035,16/01/2023,23/04/2025,...,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114082-rec...
621,CONSULTANT FORMATEUR EN DROIT,"Description du posteNous, cabinet de formation...",86696,27/07/2022,02/03/2024,ABIDJAN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114081-lab...
622,CONSULTANT FORMATEUR SPECIALISE,"Description du posteNous, cabinet de formation...",86695,27/07/2022,02/03/2024,ABIDJAN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-114080-con...


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
import time

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


# (votre fonction extract_job_information reste inchangée)
def extract_job_information(soup, url):
    try:
        # Extraction des informations de l'offre d'emploi
        poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
        type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
        metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
        niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
        experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
        lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
        
        # Extraction des dates de publication et de limite
        date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
        date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
        
        # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
        description = soup.select_one('div.entry-content').text.strip()

        return {
            "Poste": [poste],
            "Type d'offre": [type_offre],
            "Métier(s)": [metiers],
            "Niveau(x)": [niveaux],
            "Expérience": [experience],
            "Lieu": [lieu],
            "Offre_Link": [url],
            "Date de publication": [date_publication],
            "Date limite": [date_limite],
            "Description": [description]
        }
    except Exception as e:
        print(f"An error occurred while extracting job information for URL {url}: {e}")
        # Retourner un dictionnaire avec l'URL en cas d'erreur
        return {"Offre_Link": [url]}

# Liste des URLs à scraper
urls = list(result_df['Offre_Link'])

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur chaque URL
for url in urls:
    try:
        # Envoyer une requête GET au site avec un délai de 120 secondes
        response = requests.get(url, headers=headers, verify=True, timeout=120)

        # Vérifier si la requête a réussi (statut 200)
        if response.status_code == 200:
            # Analyser le contenu de la page avec BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire les informations sur l'emploi
            job_info = extract_job_information(soup, url)

            # Créer un DataFrame
            df = pd.DataFrame(job_info)
            #df['URLL']=url

            # Ajouter le DataFrame à la liste
            dfs.append(df)
        else:
            print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))

    except requests.exceptions.Timeout:
        print(f"Timeout lors de la requête pour l'URL {url}")
        # Ajouter une ligne avec l'URL en cas d'erreur
        dfs.append(pd.DataFrame({"Offre_Link": [url]}))
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")
        # Ajouter une ligne avec l'URL en cas d'erreur
        dfs.append(pd.DataFrame({"Offre_Link": [url]}))

    # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
    time.sleep(5)

# Concaténer tous les DataFrames en un seul DataFrame
df_Educarriere = pd.concat(dfs, ignore_index=True)
df_Educarriere

,Poste,Type d'offre,Métier(s),Niveau(x),Expérience,Lieu,URL,Date de publication,Date limite,Description
0,INGENIEUR MODELISATION RESEAU,Emploi,"Agriculture, Informatique, Maintenance Informa...",BAC+5,4 ans,Côte d'Ivoire,https://emploi.educarriere.ci/offre-114121-ing...,14/02/2024,26/02/2024,AFRIJOBrecruteINGENIEUR MODELISATION RESEAU\n ...
1,AGENT MARKETING ET COMMUNICATION,Stage,Marketing,"BAC+3, BAC+5",,Côte d'Ivoire,https://emploi.educarriere.ci/offre-114120-age...,14/02/2024,26/02/2024,WAGSystemsrecruteAGENT MARKETING ET COMMUNICAT...
2,VISA SPECIALIST,Stage,Ressources Humaines,BAC+3,,Côte d'Ivoire,https://emploi.educarriere.ci/offre-114119-vis...,14/02/2024,26/02/2024,HUAWEI TECHNOLOGIES COTE D'IVOIRE SAUrecruitVI...
3,PLIEUR,Emploi,Chaudronnerie,BT,3 ans,FRANCE,https://emploi.educarriere.ci/offre-114118-pli...,14/02/2024,29/02/2024,AGENCE IVOIRE INTERIM (A2I)recrutePLIEUR\n \n ...
4,COMMERCIAL,Stage,"Commerce et Administration des Entreprises, Ma...",BAC+2,,ABIDJAN,https://emploi.educarriere.ci/offre-114117-com...,14/02/2024,29/02/2024,NISSIM ConsultingrecruteCOMMERCIAL\n \n \nDesc...
...,...,...,...,...,...,...,...,...,...,...
580,Junior Mudlogger - Field Engineer (analyse et ...,Emploi,Mines/Géologie/Pétrole,"BAC+3, BAC+4, BAC+5",,Abidjan,https://emploi.educarriere.ci/offre-114083-jun...,14/02/2024,29/02/2024,GEOLOG FRANCE SUCCURSALE CIrecruteJunior Mudlo...
581,Une secrétaire stagiaire,Stage,Secrétariat,BAC+2,1 ans,Poste basé dans la commune de Cocody,https://emploi.educarriere.ci/offre-114082-rec...,14/02/2024,26/02/2024,BEST SARLrecruteUne secrétaire stagiaire\n \n ...
582,LABORANTIN STAGIAIRE,Stage,"Chimie, Qualité",BAC+2,,Grand Zattry (Soubré),https://emploi.educarriere.ci/offre-114081-lab...,14/02/2024,23/02/2024,SOUTHLAND RESSOURCES CIrecruteLABORANTIN STAGI...
583,CONDUCTEUR PROFESSIONNEL,Emploi,"Génie Mécanique et Productique, Mécanique, Méc...","BT, BAC, BAC+2, BAC+3",3 ans,Côte d'Ivoire,https://emploi.educarriere.ci/offre-114080-con...,14/02/2024,29/02/2024,KASSA GOLD SArecruteCONDUCTEUR PROFESSIONNEL\n...


In [ ]:
# Fusionner les deux DataFrames sur la colonne 'URL'
Educarriere_df = pd.merge(result_df, df_Educarriere, on='Offre_Link')

# Afficher le DataFrame fusionné
Educarriere_df    
    

In [5]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_Educarriere['Poste'] = list(result_df['Poste'])
df_Educarriere['Sous_titre'] = list(result_df['Sous_titre'])
df_Educarriere['Code'] = list(result_df['Code'])
df_Educarriere['Date_DEdition'] = list(result_df['Date_DEdition'])
df_Educarriere['Date_limite'] = list(result_df['Date_limite'])
df_Educarriere['Pays'] = list(result_df['Pays'])



# Réorganiser les colonnes selon vos besoins
df_Educarriere 

,Poste,Type d'offre,Métier(s),Niveau(x),Expérience,Lieu,URL,Date de publication,Date limite,Description,Sous_titre,Code,Date_DEdition,Date_limite,Pays
0,DECLARANT EN DOUANE,Emploi,"Agriculture, Informatique, Maintenance Informa...",BAC+5,4 ans,Côte d'Ivoire,https://emploi.educarriere.ci/offre-114121-ing...,14/02/2024,26/02/2024,AFRIJOBrecruteINGENIEUR MODELISATION RESEAU\n ...,EXCELIAMrecruteDECLARANT EN DOUANEDescription ...,109945,14/02/2024,14/03/2024,d'I...
1,INGENIEUR MODELISATION RESEAU,Stage,Marketing,"BAC+3, BAC+5",,Côte d'Ivoire,https://emploi.educarriere.ci/offre-114120-age...,14/02/2024,26/02/2024,WAGSystemsrecruteAGENT MARKETING ET COMMUNICAT...,AFRIJOBrecruteINGENIEUR MODELISATION RESEAUDes...,109944,14/02/2024,26/02/2024,d'I...
2,AGENT MARKETING ET COMMUNICATION,Stage,Ressources Humaines,BAC+3,,Côte d'Ivoire,https://emploi.educarriere.ci/offre-114119-vis...,14/02/2024,26/02/2024,HUAWEI TECHNOLOGIES COTE D'IVOIRE SAUrecruitVI...,WAGSystemsrecruteAGENT MARKETING ET COMMUNICAT...,109943,14/02/2024,26/02/2024,d'I...
3,VISA SPECIALIST,Emploi,Chaudronnerie,BT,3 ans,FRANCE,https://emploi.educarriere.ci/offre-114118-pli...,14/02/2024,29/02/2024,AGENCE IVOIRE INTERIM (A2I)recrutePLIEUR\n \n ...,HUAWEI TECHNOLOGIES COTE D'IVOIRE SAUrecruitVI...,109942,14/02/2024,26/02/2024,d'I...
4,PLIEUR,Stage,"Commerce et Administration des Entreprises, Ma...",BAC+2,,ABIDJAN,https://emploi.educarriere.ci/offre-114117-com...,14/02/2024,29/02/2024,NISSIM ConsultingrecruteCOMMERCIAL\n \n \nDesc...,AGENCE IVOIRE INTERIM (A2I)recrutePLIEURDescri...,109941,14/02/2024,29/02/2024,FRANCE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,STAGE DE SOUTENANCE (RH-COM; GESCOM; FCGE; RIT...,Emploi,Mines/Géologie/Pétrole,"BAC+3, BAC+4, BAC+5",,Abidjan,https://emploi.educarriere.ci/offre-114083-jun...,14/02/2024,29/02/2024,GEOLOG FRANCE SUCCURSALE CIrecruteJunior Mudlo...,Description du posteStage de soutenanceNous of...,95492,14/02/2023,27/03/2024,...
581,ESTHÉTICIENNE - PROTHÉSISTE ONGULAIRE (H/F),Stage,Secrétariat,BAC+2,1 ans,Poste basé dans la commune de Cocody,https://emploi.educarriere.ci/offre-114082-rec...,14/02/2024,26/02/2024,BEST SARLrecruteUne secrétaire stagiaire\n \n ...,Description du posteNous recherchons des esthé...,94035,16/01/2023,23/04/2025,...
582,CONSULTANT FORMATEUR EN DROIT,Stage,"Chimie, Qualité",BAC+2,,Grand Zattry (Soubré),https://emploi.educarriere.ci/offre-114081-lab...,14/02/2024,23/02/2024,SOUTHLAND RESSOURCES CIrecruteLABORANTIN STAGI...,"Description du posteNous, cabinet de formation...",86696,27/07/2022,02/03/2024,ABIDJAN
583,CONSULTANT FORMATEUR SPECIALISE,Emploi,"Génie Mécanique et Productique, Mécanique, Méc...","BT, BAC, BAC+2, BAC+3",3 ans,Côte d'Ivoire,https://emploi.educarriere.ci/offre-114080-con...,14/02/2024,29/02/2024,KASSA GOLD SArecruteCONDUCTEUR PROFESSIONNEL\n...,"Description du posteNous, cabinet de formation...",86695,27/07/2022,02/03/2024,ABIDJAN


# NOVOJOB

In [34]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Liste des liens pour chaque catégorie
categories = [
    "toutes les offres d'emploi",
    "juridique,fiscal,audit,conseil",
    "administrations,moyens généraux",
    "assistanat,secrétariat",
    "metiers banque et assurances",
    "RH,personnel,formation",
    "education,enseignement",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",
    "Métiers de l'agriculture"
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
niveau_list = []
experience_list = []

url_list = []
all_job_lien = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    for offre in offres:
        offre_link_tag = offre.find('a')
        if offre_link_tag:
            offre_link = offre_link_tag['href']
            all_job_lien.append(offre_link)

        entreprise_tag = offre.find('h6', class_='ellipsis')
        entreprise = entreprise_tag.get_text().strip() if entreprise_tag else None

        intitule_tag = offre.find('h2', class_='ellipsis row-fluid')
        intitule = intitule_tag.get_text().strip() if intitule_tag else None

        bloc_bottom = offre.find_next('div', class_='bloc-bottom')

        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None

        niveau_info = bloc_bottom.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        match = re.match(r'(.+) \((.+)\)', niveau_text)
        niveau_col, experience_col = match.groups() if match else (None, None)

        intitules_list.append(intitule)
        entreprises_list.append(entreprise)
        pays_list.append(pays)
        dates_list.append(date)
        niveau_list.append(niveau_col)
        experience_list.append(experience_col)
        
        url_list.append(category_link)

# Création du DataFrame
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'url': url_list,
    'Offre_Link': all_job_lien
})


df_offers

,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,url,Offre_Link
0,Livreurs Moto,Entreprise anonyme,Côte d'ivoire,04 Janvier,Confirmé / Expérimenté,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
1,Pompiste H/F,None,"Abidjan, Côte d'ivoire",27 Décembre 2023,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
2,Caissier H/F,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
3,Ouvrier de rayon,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
4,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
...,...,...,...,...,...,...,...,...
706,Responsable Clientèle Patrimoniale (H/F),Société Ivoirienne de Banque (SIB),"Abidjan, Côte d'ivoire",05 Février,Confirmé / Expérimenté,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
707,Technicien Génie Civil Option Bâtiment,SOCOPI,Côte d'ivoire,17 Janvier,Débutant / Junior,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
708,Chargé de Support Système Windows et Virtualis...,Ascens,"Abidjan, Côte d'ivoire",24 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
709,Chargé.e de Dossiers Achats et Contrats (Assis...,Giz Côte d'Ivoire,"Korhogo, Côte d'ivoire",01 Février,Débutant / Junior,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...


In [35]:

import requests
import pandas as pd

# Example usage for multiple job URLs
job_urls = list(df_offers['Offre_Link'])

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}
    
    # Ajouter le lien
    job_details["Offre_Link"]=url
    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    all_job_details.append(job_details)

# Create a DataFrame
df_Novojob = pd.DataFrame(all_job_details)
df_Novojob


,Offre_Link,Nom de l'entreprise,Secteur d'activité,Lieu de travail,Date d'expiration,Nombre de postes,Niveau de poste,Niveau d'étude (diplome),Type de contrat,Provided Text
0,https://www.novojob.com/cote-d-ivoire/offres-d...,Entreprise anonyme,Services,Côte d'ivoire,03 Avril,40 postes ouverts,Débutant / Junior| Stagiaire / Etudiant| Confi...,Niveau secondaire| Niveau terminal| Baccalauré...,Mission,SPPCI Offre d'emploiLivreurs Moto (H/F)Vous êt...
1,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Abidjan, Côte d'ivoire",19 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...
2,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,Côte d'ivoire,26 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...
3,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,Côte d'ivoire,26 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...
4,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Informatique, Télécom, Internet",Côte d'ivoire,02 Mai,25,Débutant / Junior| Stagiaire / Etudiant| Jeune...,"TS Bac +2| Licence (LMD), Bac + 3",Consultant,KOMIAN IA est une entreprise de technologie d’...
...,...,...,...,...,...,...,...,...,...,...
706,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Banque, Assurance, Finance","Abidjan, Côte d'ivoire",05 Mai,01,Confirmé / Expérimenté,"Master 1, Licence Bac + 4| Master 2, Ingénior...",CDI,"Présentation de l’Entreprise La SIB, Société I..."
707,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"BTP, Construction, Immobilier",Côte d'ivoire,16 Avril,01,Débutant / Junior,"Master 2, Ingéniorat, Bac + 5",CDD,SOCOPI recrute un technicien génie civil optio...
708,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Informatique, Télécom, Internet","Abidjan, Côte d'ivoire",22 Février,01,Confirmé / Expérimenté,"Master 2, Ingéniorat, Bac + 5",CDI,ASCENS SERVICES est une filiale du Groupe Afri...
709,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Fonction publique, Administration","Korhogo, Côte d'ivoire",22 Février,01,Débutant / Junior,TS Bac +2,CDD,Poste basé à KorhogoProjet : ...


In [ ]:
# Fusionner les deux DataFrames sur la colonne 'URL'
Novojob_df = pd.merge(df_offers, df_Novojob, on='Offre_Link')

# Afficher le DataFrame fusionné
Novojob_df    
    

In [37]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_Novojob['Intitule'] = intitules_list
df_Novojob['Entreprise'] = entreprises_list
df_Novojob['Pays'] = pays_list
df_Novojob['Date'] = dates_list
df_Novojob['Niveau'] = niveau_list
df_Novojob['Experience_lettre'] = experience_list
df_Novojob['url_Lien'] = url_list
df_Novojob['Offre_Link1']= all_job_lien

# Réorganiser les colonnes selon vos besoins

df_Novojob

,Offre_Link,Nom de l'entreprise,Secteur d'activité,Lieu de travail,Date d'expiration,Nombre de postes,Niveau de poste,Niveau d'étude (diplome),Type de contrat,Provided Text,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,url_Lien,Offre_Link1
0,https://www.novojob.com/cote-d-ivoire/offres-d...,Entreprise anonyme,Services,Côte d'ivoire,03 Avril,40 postes ouverts,Débutant / Junior| Stagiaire / Etudiant| Confi...,Niveau secondaire| Niveau terminal| Baccalauré...,Mission,SPPCI Offre d'emploiLivreurs Moto (H/F)Vous êt...,Livreurs Moto,Entreprise anonyme,Côte d'ivoire,04 Janvier,Confirmé / Expérimenté,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
1,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Abidjan, Côte d'ivoire",19 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...,Pompiste H/F,None,"Abidjan, Côte d'ivoire",27 Décembre 2023,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
2,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,Côte d'ivoire,26 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...,Caissier H/F,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
3,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,Côte d'ivoire,26 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...,Ouvrier de rayon,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
4,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Informatique, Télécom, Internet",Côte d'ivoire,02 Mai,25,Débutant / Junior| Stagiaire / Etudiant| Jeune...,"TS Bac +2| Licence (LMD), Bac + 3",Consultant,KOMIAN IA est une entreprise de technologie d’...,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Banque, Assurance, Finance","Abidjan, Côte d'ivoire",05 Mai,01,Confirmé / Expérimenté,"Master 1, Licence Bac + 4| Master 2, Ingénior...",CDI,"Présentation de l’Entreprise La SIB, Société I...",Responsable Clientèle Patrimoniale (H/F),Société Ivoirienne de Banque (SIB),"Abidjan, Côte d'ivoire",05 Février,Confirmé / Expérimenté,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
707,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"BTP, Construction, Immobilier",Côte d'ivoire,16 Avril,01,Débutant / Junior,"Master 2, Ingéniorat, Bac + 5",CDD,SOCOPI recrute un technicien génie civil optio...,Technicien Génie Civil Option Bâtiment,SOCOPI,Côte d'ivoire,17 Janvier,Débutant / Junior,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
708,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Informatique, Télécom, Internet","Abidjan, Côte d'ivoire",22 Février,01,Confirmé / Expérimenté,"Master 2, Ingéniorat, Bac + 5",CDI,ASCENS SERVICES est une filiale du Groupe Afri...,Chargé de Support Système Windows et Virtualis...,Ascens,"Abidjan, Côte d'ivoire",24 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
709,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Fonction publique, Administration","Korhogo, Côte d'ivoire",22 Février,01,Débutant / Junior,TS Bac +2,CDD,Poste basé à KorhogoProjet : ...,Chargé.e de Dossiers Achats et Contrats (Assis...,Giz Côte d'Ivoire,"K

In [42]:
# Exportez le DataFrame en fichier CSV
df_Novojob.to_excel("df_Novojob.xlsx", index=False)

# Emploi.ci

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    tag = element.find(tag_name)
    return tag.text.strip() if tag else ""

def clean_text(text):
    return text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')

def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='job-description-wrapper')

    data_list = []

    for wrapper in job_description_wrappers:
        h5_tag = wrapper.find('h5')
        poste = extract_text(h5_tag, 'a')

        job_recruiter_tag = wrapper.find('p', class_='job-recruiter')
        date_and_company = job_recruiter_tag.text.strip().split('|')
        date = date_and_company[0].strip() if date_and_company else ""
        entreprise = extract_text(job_recruiter_tag, 'a')

        description_tag = wrapper.find('div', class_='search-description')
        description = clean_text(description_tag.text.strip()) if description_tag else ""

        region_tag = wrapper.find('p', text='Région de :')
        region = extract_text(region_tag) if region_tag else ""

        data_list.append({
            'Poste': poste,
            'Entreprise': entreprise,
            'Date': date,
            'Description': description,
            'Région': region,
            'URL' : url
        })

    df = pd.DataFrame(data_list)
    return df

# Liste des liens
categories = ["31", "1127", "29", "37", "1115", "30", "1115", "32", "33", "34", "35", "36", "37", "39", "38", "40", "525", "41", "28"]
#categories=["31"]
# Liste d'URLs générées
urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]

# Créer un DataFrame à partir des liens
df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Afficher le DataFrame
df


C:\Users\ngora\AppData\Local\Temp\ipykernel_19868\3728830876.py:38: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  region_tag = wrapper.find('p', text='Région de :')


,Poste,Entreprise,Date,Description,Région,URL
0,Consultant Senior SIRH,AFRICAWORK,23.02.2024,Nous recherchons notre futur(e) Consultant Sen...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...
1,Manager Stratégie IT & Digitale,AFRICAWORK,23.02.2024,Nous recherchons notre futur(e) Manager Straté...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...
2,Consultant Expérimente Sage (Démarrage ASAP),AFRICAWORK,23.02.2024,Nous recherchons notre futur(e) Consultant Exp...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...
3,Sharepoint Developper Intern,ENDEAVOUR MINING,23.02.2024,Nous sommes à la recherche d'un Stagiaire Shar...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...
4,Stagiaire Designer Télécom H/F,REIME CÔTE D'IVOIRE,20.02.2024,Si vous êtes passionné(e) par les technologies...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...
...,...,...,...,...,...,...
229,Logistics and Importation Coordinator,ALDELIA,16.02.2024,Description du posteLe coordinateur logistique...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...
230,Chauffeur Intérimaire H/F,REIME CÔTE D'IVOIRE,15.02.2024,"Si vous êtes passionné(e) par la conduite, que...",,https://www.emploi.ci/recherche-jobs-cote-ivoi...
231,Responsable Service Généraux,UMO INTÉRIM,07.02.2024,SOCIÉTÉNotre client est une société internatio...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...
232,Assembleur Flexible,NEEMBA,02.02.2024,Mission principale:Être le garant des activité...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...


In [2]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens

# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens
for url in list(df["URL"]):
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_="job-description-wrapper")

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <h4> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('h5')

        # Vérifier si la balise <h4> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag.find('a')['href']
            all_job_details.append({'Offre_Link': "https://www.emploi.ci"+offre_link, 'URL' :url})

# Fermer le pilote Selenium à la fin
driver.quit()

# Concaténer tous les détails des emplois en un seul DataFrame
if all_job_details:
    all_job_details_df = pd.DataFrame(all_job_details)
    # Afficher le DataFrame
    #print(all_job_details_df)
else:
    print("Aucun détail d'offre d'emploi trouvé.")
    
# Fusionner les deux DataFrames sur la colonne 'URL'
emploi_df = pd.merge(df, all_job_details_df, on='URL')

# Afficher le DataFrame fusionné
emploi_df    
    

,Poste,Entreprise,Date,Description,Région,URL,Offre_Link
0,Consultant Senior SIRH,AFRICAWORK,23.02.2024,Nous recherchons notre futur(e) Consultant Sen...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
1,Consultant Senior SIRH,AFRICAWORK,23.02.2024,Nous recherchons notre futur(e) Consultant Sen...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
2,Consultant Senior SIRH,AFRICAWORK,23.02.2024,Nous recherchons notre futur(e) Consultant Sen...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
3,Consultant Senior SIRH,AFRICAWORK,23.02.2024,Nous recherchons notre futur(e) Consultant Sen...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
4,Consultant Senior SIRH,AFRICAWORK,23.02.2024,Nous recherchons notre futur(e) Consultant Sen...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
...,...,...,...,...,...,...,...
113019,Warehouse Inventory Coordinator,ENDEAVOUR MINING,16.01.2024,"Reporting to Supply Chain Manager, the Warehou...",,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
113020,Warehouse Inventory Coordinator,ENDEAVOUR MINING,16.01.2024,"Reporting to Supply Chain Manager, the Warehou...",,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
113021,Warehouse Inventory Coordinator,ENDEAVOUR MINING,16.01.2024,"Reporting to Supply Chain Manager, the Warehou...",,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
113022,Warehouse Inventory Coordinator,ENDEAVOUR MINING,16.01.2024,"Reporting to Supply Chain Manager, the Warehou...",,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ChunkedEncodingError, ConnectionError, ReadTimeout

# Fonction pour extraire les informations d'une page
def extract_information(url):
    try:
        response = requests.get(url, timeout=120)  # Augmentation du délai à 20 secondes
        response.raise_for_status()
        response.encoding = 'utf-8'

        soup = BeautifulSoup(response.content, 'html.parser')

        # Extraction des informations sur l'entreprise
        company_info = soup.select_one('.job-ad-company')
        entreprise = {
            "offre_url" : url,
            'Nom': company_info.select_one('.company-title a').text.strip() if company_info and company_info.select_one('.company-title a') else None,
            'Secteur d´activité': ', '.join(item.text.strip() for item in company_info.select('.sector-title .field-item')) if company_info and company_info.select('.sector-title .field-item') else None,
            'Description de l\'entreprise': soup.select_one('.job-ad-company-description label + *').text.strip() if soup.select_one('.job-ad-company-description label + *') else None
        }

        # Extraction des informations sur l'annonce
        annonce_info = soup.select_one('.job-ad-details')
        annonce = {
            'Poste': soup.select_one('.ad-ss-title').text.strip() if soup.select_one('.ad-ss-title') else None,
            'Missions': [li.text.strip() for li in soup.select('.content ul.missions li')] if soup.select('.content ul.missions') else None,
            'Profil recherché': [li.text.strip() for li in soup.select('.content ul.profil li')] if soup.select('.content ul.profil') else None,
            'Métier': soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item') else None,
            'Secteur d´activité (de l\'annonce)': soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item') else None,
            'Type de contrat': soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item') else None,
            'Région': soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item') else None,
            'Ville': soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item') else None,
            'Niveau d\'expérience': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item') else None,
            'Niveau d\'études': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item') else None,
            'Compétences clés': [li.text.strip() for li in soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item')] if soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item') else None,
            'Nombre de poste(s)': soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td').text.strip() if soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td') else None,
}

        return {'entreprise': entreprise, 'annonce': annonce}

    except (ConnectionError, ReadTimeout, ChunkedEncodingError) as e:
        print(f"Erreur lors de la requête {url}: {e}")
        # Relancer la requête
        entreprise = {
            "offre_url" : url,
            'Nom': "",
            'Secteur d´activité': "",
            'Description de l\'entreprise':""}
        annonce = {'Poste':"",
                   'Missions': "",
                   'Profil recherché':"",
                   'Métier':"",
                   'Secteur d´activité (de l\'annonce)':"",
                   'Type de contrat':"",
                   'Région': "",
                   'Ville':"",
                   'Niveau d\'expérience':"",
                   'Niveau d\'études':"",
                   'Compétences clés':"",
                   'Nombre de poste(s)':""}
                   
                   
                   
        return {'entreprise': entreprise, 'annonce': annonce}

# Liste des URLs
urls = list(emploi_df['Offre_Link'])

# Initialisation d'une liste pour stocker les DataFrames
df_list = []

# Boucle à travers chaque URL
for url in urls:
    data = extract_information(url)

    # Si la requête a échoué, passez à l'URL suivante
    if data is None:
        continue

    # Création du DataFrame pour chaque URL
    df = pd.DataFrame([data['entreprise'] | data['annonce']])

    # Ajout du DataFrame à la liste
    df_list.append(df)

# Concaténation des DataFrames de chaque URL
result_df = pd.concat(df_list, ignore_index=True)
result_df


C:\Users\ngora\anaconda3\Lib\site-packages\soupsieve\css_parser.py:872: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(


Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/devops-engineer-mf-1386340: HTTPSConnectionPool(host='www.emploi.ci', port=443): Read timed out.
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/dotnet-developer-mf-1386341: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/dotnet-developer-mf-1386341 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF3F4A2E50>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/ios-developer-mf-1386342: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/ios-developer-mf-1386342 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF3F2BB650>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête http

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/sharepoint-developper-intern-1389480: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/sharepoint-developper-intern-1389480 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF3FE0E890>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF40A152D0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-experimente-cyber-risk-fh-1387731: HTTPSConnectionPool(host='www.emploi.

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/test-automation-engineer-mf-1386357: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/test-automation-engineer-mf-1386357 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF3C42A4D0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-senior-sirh-1389385: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/consultant-senior-sirh-1389385 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF4039C050>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/manager-strategie-it-digitale-1389390: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max ret

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/qa-tester-mf-1386352: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/qa-tester-mf-1386352 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF403D19D0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/react-developer-mf-1386353: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/react-developer-mf-1386353 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF406A84D0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/ruby-on-rails-developer-mf-1386354: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cot

...

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/javascript-developer-mf-1386345: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/javascript-developer-mf-1386345 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF40A12890>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/kotlin-developer-mf-1386346: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/kotlin-developer-mf-1386346 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF3FF2F550>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/laravel-developer-mf-1386347: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url:

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/devops-engineer-mf-1386340: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/devops-engineer-mf-1386340 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF3FFDAB90>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/dotnet-developer-mf-1386341: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/dotnet-developer-mf-1386341 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF40A15B50>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/ios-developer-mf-1386342: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/sharepoint-developper-intern-1389480: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/sharepoint-developper-intern-1389480 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF40A12950>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF4010A650>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-experimente-cyber-risk-fh-1387731: HTTPSConnectionPool(host='www.emploi.

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/test-automation-engineer-mf-1386357: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/test-automation-engineer-mf-1386357 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF3FF7E650>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-senior-sirh-1389385: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/consultant-senior-sirh-1389385 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF40B9A9D0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/manager-strategie-it-digitale-1389390: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max ret

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/qa-tester-mf-1386352: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/qa-tester-mf-1386352 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF3F1AF150>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))


# agenceemploijeunes.ci

In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# Liste des URLs à scraper
urls = ["https://agenceemploijeunes.ci/site/offres-emplois?page={}".format(category) for category in range(17)]

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Listes pour stocker les données
job_titles = []
publication_dates = []
application_deadlines = []
locations = []
job_descriptions = []
job_types = []
diploma_requirements = []
url_lien = []

# Loop à travers chaque URL
for url in urls:
    # Envoyer une requête au site web
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    # Trouver les annonces d'emploi
    job_listings = soup.find_all('div', class_='post-bx')

    # Extract data from each job listing
    for job_listing in job_listings:
        url_lien.append(url)
        
        # Titre du poste
        job_title = job_listing.find('h4').text.strip()
        job_titles.append(job_title)

        # Dates de publication et de candidature
        date_info = job_listing.find_all('li', {'class': ''})
        if date_info:
            publication_date = date_info[0].text.replace('Publié le:', '').strip()
            application_deadline = date_info[1].text.replace('Date limite:', '').strip()
            publication_dates.append(publication_date)
            application_deadlines.append(application_deadline)

        # Localisation
        location = date_info[2].text.replace('ABENGOUROU', '').strip()
        locations.append(location)

        # Description du poste
        job_description = job_listing.find('p').text.strip()
        job_descriptions.append(job_description)

        # Type de poste
        job_type = job_listing.find('span', {'class': 'pull-right'}).text.strip()
        job_types.append(job_type)

        # Exigences en diplôme
        diploma_requirement = job_listing.find('div', {'class': 'salary-bx'}).text.strip()
        diploma_requirements.append(diploma_requirement)


# Créer un DataFrame Pandas
data = {
    'Job Title': job_titles,
    'Publication Date': publication_dates,
    'Application Deadline': application_deadlines,
    'Location': locations,
    'Job Description': job_descriptions,
    'Job Type': job_types,
    'Diploma Requirement': diploma_requirements,
    "URL": url_lien
}

df_agenceemploijeunes = pd.DataFrame(data)

# Utiliser la méthode str.extract pour extraire la valeur après "Diplôme :"
df_agenceemploijeunes['Diplome'] = df_agenceemploijeunes['Diploma Requirement'].str.extract(r'Diplôme :[ \t]([^\n\r])')


# Scrapper les détails supplémentaires à partir des URL
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# List to store job details
all_job_details = []

# Parcourir les liens
for url in urls:
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)

    # Trouver toutes les offres d'emploi sur la page
    offer_links = soup.select('.post-bx h4 a')

    # Parcourir les liens d'offres
    for offer_link in offer_links:
        # Extraire l'URL de l'offre
        offer_url = offer_link.get('href')
        
        # Ajouter les détails à la liste
        all_job_details.append({'Offre_Link': offer_url, 'URL': url})

# Fermer le navigateur à la fin
driver.quit()

# Créer un DataFrame avec les détails des offres d'emploi
if all_job_details:
    all_job_details_df = pd.DataFrame(all_job_details)
else:
    print("Aucun détail d'offre d'emploi trouvé.")

# Fusionner les deux DataFrames sur la colonne 'URL'
df_agenceemploi_jeunes = pd.merge(df_agenceemploijeunes, all_job_details_df, on='URL')

# Afficher le DataFrame fusionné
df_agenceemploi_jeunes

,Job Title,Publication Date,Application Deadline,Location,Job Description,Job Type,Diploma Requirement,URL,Diplome,Offre_Link
0,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...
1,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...
2,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...
3,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...
4,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...
...,...,...,...,...,...,...,...,...,...,...
1295,TECHNICIEN BATIMENT ET TRAVAUX PUBLICS,07 12 2023,29 02 2024,Marcory Bi.,,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...
1296,TECHNICIEN BATIMENT ET TRAVAUX PUBLICS,07 12 2023,29 02 2024,Marcory Bi.,,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...
1297,TECHNICIEN BATIMENT ET TRAVAUX PUBLICS,07 12 2023,29 02 2024,Marcory Bi.,,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...
1298,TECHNICIEN BATIMENT ET TRAVAUX PUBLICS,07 12 2023,29 02 2024,Marcory Bi.,,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...


In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Création d'un dictionnaire pour stocker les données
job_data = {
    'Job Title': [],
    'Location': [],
    'Reference': [],
    'Number of Positions': [],
    'Closing Date': [],
    'Diploma': [],
    'Job Type': [],
    'Experience': [],
    'Education Level': [],
    'Gender': [],
    'Job Description': [],
    'Offre_Link': []
}

# Loop through each URL
for url in df_agenceemploi_jeunes["Offre_Link"]:
    offre_url = url  # Sauvegarder l'URL même en cas d'exception
    try:
        # Send a request to the website
        req = requests.get(url, headers=headers)
        req.raise_for_status()  # Raise an error for unsuccessful responses
        soup = BeautifulSoup(req.text, 'html.parser')

        # Extract job details
        job_details = soup.find('div', class_='widget_getintuch')

        if job_details:
            # Extract data from job details
            ul_element = job_details.find('ul')
            if ul_element:
                details_list = ul_element.find_all('li')

                # Initialize variables to store details
                location = reference = num_positions = closing_date = diploma = job_type = experience = education_level = gender = None

                # Iterate through details
                for detail in details_list:
                    label = detail.find('strong')
                    value_span = detail.find('span', class_='text-black-light')

                    if label and value_span:
                        label_text = label.text.strip()
                        value_text = value_span.text.strip()

                        if 'Lieu de travail' in label_text:
                            location = value_text
                        elif 'Reference' in label_text:
                            reference = value_text
                        elif 'Nombre de poste' in label_text:
                            num_positions = value_text
                        elif 'Date de clôture' in label_text:
                            closing_date = value_text
                        elif 'Diplôme' in label_text:
                            diploma = value_text
                        elif 'Type de contrat' in label_text:
                            job_type = value_text
                        elif 'Expérience professionnelle' in label_text:
                            experience = value_text
                        elif 'Niveau d\'études' in label_text:
                            education_level = value_text
                        elif 'Sexe' in label_text:
                            gender = value_text

                # Append extracted details to the dictionary
                job_data['Location'].append(location)
                job_data['Reference'].append(reference)
                job_data['Number of Positions'].append(num_positions)
                job_data['Closing Date'].append(closing_date)
                job_data['Diploma'].append(diploma)
                job_data['Job Type'].append(job_type)
                job_data['Experience'].append(experience)
                job_data['Education Level'].append(education_level)
                job_data['Gender'].append(gender)

            # Extract job title and description
            job_title_element = soup.find('h3', {'class': 'title-head'})
            if job_title_element:
                job_title = job_title_element.text.strip()
                job_data['Job Title'].append(job_title)

                job_description_info = soup.find('div', {'class': 'job-info-box'}).find('ul')
                if job_description_info:
                    job_description_text = '\n'.join([li.text.strip() for li in job_description_info.find_all('li')])
                    job_data['Job Description'].append(job_description_text)
                else:
                    job_data['Job Description'].append(None)
            else:
                job_data['Job Title'].append(None)
                job_data['Job Description'].append(None)
        else:
            job_data['Job Title'].append(None)
            job_data['Job Description'].append(None)

        # Append URL to the dictionary
        job_data['Offre_Link'].append(offre_url)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while accessing URL: {url}")
        print(e)
        # Ajouter l'URL même en cas d'exception
        job_data['Location'].append(None)
        job_data['Reference'].append(None)
        job_data['Number of Positions'].append(None)
        job_data['Closing Date'].append(None)
        job_data['Diploma'].append(None)
        job_data['Job Type'].append(None)
        job_data['Experience'].append(None)
        job_data['Education Level'].append(None)
        job_data['Gender'].append(None)
        job_data['Job Title'].append(None)
        job_data['Job Description'].append(None)
        job_data['Offre_Link'].append(offre_url)

# Create DataFrame from the collected data
df_jobs = pd.DataFrame(job_data)
df_jobs


An error occurred while accessing URL: https://agenceemploijeunes.ci/site/offres-emplois/33671
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred while accessing URL: https://agenceemploijeunes.ci/site/offres-emplois/33652
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred while accessing URL: https://agenceemploijeunes.ci/site/offres-emplois/33637
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred while accessing URL: https://agenceemploijeunes.ci/site/offres-emplois/33200
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


,Job Title,Location,Reference,Number of Positions,Closing Date,Diploma,Job Type,Experience,Education Level,Gender,Job Description,Offre_Link
0,COMMERCIAUX FREE-LANCE,ABIDJAN COCODY,ADJA-33683-02-2024,2,26/02/2024,BAC,CDD,1 ANS,TERMINALE,MASCULIN,CDD\nDate de clôture:\n ...,https://agenceemploijeunes.ci/site/offres-empl...
1,STAGIAIRE INFORMATICIEN,ODIENNE,ODIE-33674-02-2024,1,27/02/2024,ATTESTATION,STAGE DE QUALIFICATION,,BAC+2,None,STAGE DE QUALIFICATION\nDate de clôture:\n ...,https://agenceemploijeunes.ci/site/offres-empl...
2,None,None,None,None,None,None,None,None,None,None,None,https://agenceemploijeunes.ci/site/offres-empl...
3,STAGIATRE INFORMATICIEN,YAMOUSSOUKRO,YAMO-33670-02-2024,1,04/03/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...,https://agenceemploijeunes.ci/site/offres-empl...
4,OUVRIERS USINE,YAMOUSOUKRO,YAMO-33669-02-2024,150,04/03/2024,BEPC,CDD,,CM2,MASCULIN,CDD\nDate de clôture:\n ...,https://agenceemploijeunes.ci/site/offres-empl...
...,...,...,...,...,...,...,...,...,...,...,...,...
1295,SELECTION DE 2012 JEUNES DANS LE CADRE DES TRA...,"ANANDA, BONGUERA, KOTOBI, ANDE, ASSAHARA, BENG...",DIMB-32987-01-2024,100,26/02/2024,SANS DIPLOME,THIMO,,AUCUN,MASCULIN,THIMO\nDate de clôture:\n ...,https://agenceemploijeunes.ci/site/offres-empl...
1296,STAGIAIRE COMMERCIAL,KORHOGO-CREDIT ACCES,KORH-32888-01-2024,5,07/03/2024,BTS,STAGE DE VALIDATION,,BAC+2,MASCULIN,STAGE DE VALIDATION\nDate de clôture:\n ...,https://agenceemploijeunes.ci/site/offres-empl...
1297,STAGIARE COMMERCIAL,ABIDJAN,ABOB-32829-01-2024,10,27/02/2024,BTS,STAGE PEJEDEC,,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...,https://agenceemploijeunes.ci/site/offres-empl...
1298,SECRETARIAT BUREAUTIQUE,ABENGOUROU,ABEN-32819-01-2024,22,26/02/2024,BT,STAGE DE VALIDATION,,TERMINALE,MASCULIN,STAGE DE VALIDATION\nDate de clôture:\n ...,https://agenceemploijeunes.ci/site/offres-empl...


In [21]:
agenceemploi_jeunes_df = pd.merge(df_agenceemploi_jeunes, df_jobs, on='Offre_Link')
agenceemploi_jeunes_df

,Job Title_x,Publication Date,Application Deadline,Location_x,Job Description_x,Job Type_x,Diploma Requirement,URL,Diplome,Offre_Link,...,Location_y,Reference,Number of Positions,Closing Date,Diploma,Job Type_y,Experience,Education Level,Gender,Job Description_y
0,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABIDJAN COCODY,ADJA-33683-02-2024,2,26/02/2024,BAC,CDD,1 ANS,TERMINALE,MASCULIN,CDD\nDate de clôture:\n ...
1,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABIDJAN COCODY,ADJA-33683-02-2024,2,26/02/2024,BAC,CDD,1 ANS,TERMINALE,MASCULIN,CDD\nDate de clôture:\n ...
2,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABIDJAN COCODY,ADJA-33683-02-2024,2,26/02/2024,BAC,CDD,1 ANS,TERMINALE,MASCULIN,CDD\nDate de clôture:\n ...
3,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABIDJAN COCODY,ADJA-33683-02-2024,2,26/02/2024,BAC,CDD,1 ANS,TERMINALE,MASCULIN,CDD\nDate de clôture:\n ...
4,COMMERCIAUX FREE-LANCE,20 02 2024,26 02 2024,ABIDJAN CO.,VENTE DES KITS SOLAIRES \r\n\r\n-SUIVRE INTÉGR...,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABIDJAN COCODY,ADJA-33683-02-2024,2,26/02/2024,BAC,CDD,1 ANS,TERMINALE,MASCULIN,CDD\nDate de clôture:\n ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,TECHNICIEN BATIMENT ET TRAVAUX PUBLICS,07 12 2023,29 02 2024,Marcory Bi.,,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,Marcory Bietry,DG --32319-12-2023,1,29/02/2024,BTS,CDD,01 ANS MINIMUM,BAC+2,MASCULIN,CDD\nDate de clôture:\n ...
14996,TECHNICIEN BATIMENT ET TRAVAUX PUBLICS,07 12 2023,29 02 2024,Marcory Bi.,,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,Marcory Bietry,DG --32319-12-2023,1,29/02/2024,BTS,CDD,01 ANS MINIMUM,BAC+2,MASCULIN,CDD\nDate de clôture:\n ...
14997,TECHNICIEN BATIMENT ET TRAVAUX PUBLICS,07 12 2023,29 02 2024,Marcory Bi.,,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,Marcory Bietry,DG --32319-12-2023,1,29/02/2024,BTS,CDD,01 ANS MINIMUM,BAC+2,MASCULIN,CDD\nDate de clôture:\n ...
14998,TECHNICIEN BATIMENT ET TRAVAUX PUBLICS,07 12 2023,29 02 2024,Marcory Bi.,,CDD,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,Marcory Bietry,DG --32319-12-2023,1,29/02/2024,BTS,CDD,01 ANS MINIMUM,BAC+2,MASCULIN,CDD\nDate de clôture:\n ...


# Banque Mondiale

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Fonction pour extraire les données d'une page
def scrape_page(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Lèvera une exception si la requête a échoué
        soup = BeautifulSoup(response.content, "html.parser")
        job_rows = soup.find("table", class_="results-table").find("tbody").find_all("tr")

        # Listes pour stocker les données de cette page
        job_titles = []
        locations = []
        job_families = []
        deadlines = []

        # Parcourir chaque ligne du tableau et extraire les informations nécessaires
        for row in job_rows:
            # Récupérer le titre de l'emploi
            job_title = row.find("a").text.strip()
            job_titles.append(job_title)

            # Récupérer l'emplacement
            location = row.find_all("td")[1].text.strip()
            locations.append(location)

            # Récupérer la famille d'emploi
            job_family = row.find_all("td")[2].text.strip()
            job_families.append(job_family)

            # Récupérer la date limite d'application
            deadline = row.find_all("td")[3].text.strip()
            deadlines.append(deadline)

        # Retourner les données de cette page sous forme de DataFrame
        data = {
            "Job Title": job_titles,
            "Location": locations,
            "Job Family": job_families,
            "Deadline": deadlines
        }
        return pd.DataFrame(data)
    except requests.exceptions.RequestException as e:
        print("Une erreur s'est produite lors de la requête:", e)
        return pd.DataFrame()  # Retourner un DataFrame vide en cas d'erreur

# URL de la première page
base_url = "https://worldbankgroup.csod.com/ats/careersite/search.aspx?site=1&c=worldbankgroup&sid=%5e%5e%5eFLGscZMYY2RrwVaMR%2ftHYw%3d%3d"

# Créer une liste pour stocker les DataFrames de chaque page
dfs = []

# Extraire les données de la première page
dfs.append(scrape_page(base_url))

# Trouver le nombre total de pages
response = requests.get(base_url)
soup = BeautifulSoup(response.content, "html.parser")
pagination_span = soup.find("span", class_=re.compile(r"\btext\b"), text=re.compile(r"\d+"))
if pagination_span:
    num_pages = int(pagination_span.text.strip())
else:
    num_pages = 1

# Boucle à travers chaque page et extraire les données
for page_num in range(0, num_pages + 180):
    page_url = f"{base_url}&pg={page_num}"
    df = scrape_page(page_url)
    if not df.empty:  # Vérifier si le DataFrame n'est pas vide
        dfs.append(df)
# Concaténer tous les DataFrames en un seul
df_mondiale = pd.concat(dfs, ignore_index=True)

# Afficher le DataFrame final
df_mondiale


Une erreur s'est produite lors de la requête: HTTPSConnectionPool(host='worldbankgroup.csod.com', port=443): Read timed out. (read timeout=10)


C:\Users\ngora\AppData\Local\Temp\ipykernel_63656\1825849537.py:62: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  pagination_span = soup.find("span", class_=re.compile(r"\btext\b"), text=re.compile(r"\d+"))


Une erreur s'est produite lors de la requête: HTTPSConnectionPool(host='worldbankgroup.csod.com', port=443): Read timed out. (read timeout=10)
Une erreur s'est produite lors de la requête: HTTPSConnectionPool(host='worldbankgroup.csod.com', port=443): Read timed out.
Une erreur s'est produite lors de la requête: HTTPSConnectionPool(host='worldbankgroup.csod.com', port=443): Read timed out. (read timeout=10)
Une erreur s'est produite lors de la requête: HTTPSConnectionPool(host='worldbankgroup.csod.com', port=443): Read timed out.
Une erreur s'est produite lors de la requête: HTTPSConnectionPool(host='worldbankgroup.csod.com', port=443): Read timed out. (read timeout=10)
Une erreur s'est produite lors de la requête: HTTPSConnectionPool(host='worldbankgroup.csod.com', port=443): Read timed out.
Une erreur s'est produite lors de la requête: HTTPSConnectionPool(host='worldbankgroup.csod.com', port=443): Read timed out. (read timeout=10)
Une erreur s'est produite lors de la requête: HTTPSCo

,Job Title,Location,Job Family,Deadline
0,(Associate) Operations Officer,"Bogota,Colombia More",Agribusiness,2/25/2024
1,(Associate) Syndications Officer,"London,United Kingdom",Treasury,2/21/2024
2,(Senior) Industry Specialist - Real Estate Por...,"Washington, DC,United States",Carbon Finance,2/21/2024
3,Accounting Analyst - Travel,"Chennai,India",Finance & Accounting,2/21/2024
4,Associate Counsel,"Washington, DC,United States",Legal,2/29/2024
...,...,...,...,...
3275,E T Consultant,"Washington, DC,United States",Operations,2/22/2024
3276,E T Consultant,"Dhaka,Bangladesh",Environment,2/23/2024
3277,E T Consultant,"Sao Paulo,Brazil More",Agribusiness,2/23/2024
3278,E T Consultant,"Dakar,Senegal",Investment,2/22/2024


# ci.talent

In [31]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    tag = element.find(tag_name)
    return tag.text.strip() if tag else ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_talent_com(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_wrappers = soup.find_all('div', class_='card card__job')

    data_list = []

    for wrapper in job_wrappers:
        title_tag = wrapper.find('h2', class_='card__job-title')
        title = extract_text(title_tag, 'a')
        
        employer_location_tag = wrapper.find('div', class_='card__job-empnameLocation')
        #employer = extract_text(employer_location_tag.find('div', class_='card__job-location'))  # Extract location from the inner div
        location= extract_text(employer_location_tag, 'div')

        employer_location_tag = wrapper.find('div', class_='card__job-empname-label')
        
        # Extracting employer and description from the div
        employer = employer_location_tag.text.strip() if employer_location_tag else None  # Extract location from the inner div
        
        description_tag = wrapper.find('div', class_='card__job-snippet-logo')
        description = clean_text(extract_text(description_tag, 'p'))

        data_list.append({
            'Title': title,
            'Location': location,
            'Employer': employer,
            'Description': description,
            'URL': url
        })

    df = pd.DataFrame(data_list)
    return df

# List of URLs for talent.com jobs
urls = [
     "https://ci.talent.com/jobs?l=Abidjan%2C+Abidjan&radius=15&p={}&k=&context=serp_pagination".format(category) for category in range(8)
    # Add more URLs as needed "https://ci.talent.com/jobs",
]

# Initialize an empty DataFrame to store the results
combined_df = pd.DataFrame()

# Scrape job information for each URL and concatenate the results
for url in urls:
    df = scrape_talent_com(url)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Display the combined DataFrame
combined_df


,Title,Location,Employer,Description,URL
0,Country Technical Sales Manager – West Africa ...,"Abidjan, Abidjan",Claire Joster,PresentaciónClaire Joster is a headhunting fir...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
1,Global Services & Solutions Regional Coordinator,"Abidjan, Lagunes Region",myGwork,Job Description The Role Deliver a cohesive an...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
2,Conseiller national pour la mise à l'échelle (...,"Abidjan, Abidjan",Options Consultancy Services Ltd,Nous sommes une équipe mondiale d'experts et d...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
3,M&E Lead,"Abidjan, Abidjan",CARGILL,Entreprise CARGILL Site Internet. Secteur d ac...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
4,UX / UI Designer (M/F),"Abidjan, Abidjan",AFRICASHORE,Entreprise AFRICASHORE Site Internet. Secteur ...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
...,...,...,...,...,...
203,GLOBAL SOURCING MANAGER,"Abidjan, Abidjan Autonomous District",CARE,Other Possible Work Locations. Any Country whe...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
204,Chef de Projet Tuyauterie,"Abidjan, Abidjan",ORTEC Group,"À propos de nous Présent dans 25 pays, le Grou...",https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
205,Country Program Coordinator (CPC) - Ivory Coas...,"Abidjan, Abidjan Autonomous District",Employ Africa Group,Our client is a global defense solution provid...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
206,Sales Director,"Abidjan, Abidjan Autonomous District",Jelocorp,"Responsibilities Identify sales leads, pitch g...",https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...


# projobivoire

In [32]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def extract_text(element, tag_name=None):
    if element and tag_name:
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""
    return ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_projobivoire_page(page_url):
    job_data_list = []

    for url in page_url:
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            
            print(f"Erreur de connexion à {url} : {e}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        job_items = soup.find_all('div', class_='loop-item-wrap list')

        if not job_items:
            
            print(f"Aucun élément de travail trouvé pour l'URL : {url}")
            continue

        for job_item in job_items:
            title_tag = job_item.find('h3', class_='loop-item-title')
            title = extract_text(title_tag, 'a')

            job_type_tag = job_item.find('span', class_='job-type')
            job_type = extract_text(job_type_tag, 'span')

            job_date_posted = soup.find('span', class_='job-date__posted').text.strip()

            job_date_closing_tag = soup.find('span', class_='job-date__closing')
            job_date_closing = job_date_closing_tag.text.strip() if job_date_closing_tag else ""

            job_date_closing = job_date_closing.lstrip('-').strip()

            category_tag = job_item.find('span', class_='job-category')
            category = extract_text(category_tag, 'a')

            # Ajout de ces lignes pour extraire l'URL de l'e-mail
            email_url_tag = job_item.find('span', class_='noo-tool-email-job')
            email_url = email_url_tag['data-url'] if email_url_tag else ""

            data = {
                'Title': title,
                'Type': job_type,
                'DatePosted': job_date_posted,
                'DateClosing': job_date_closing,
                'Category': category,
                'EmailURL': email_url,
                'URL': url
            }

            job_data_list.append(data)

    return job_data_list

# Liste des URL de pages avec plusieurs offres d'emploi
page_urls = ["https://projobivoire.com/page/{}/".format(category) for category in range(546)]

# Scrape des détails de chaque offre d'emploi sur les pages
job_data_list = scrape_projobivoire_page(page_urls)

# Création d'un DataFrame à partir de la liste des données d'emploi
df_projobivoire = pd.DataFrame(job_data_list)

# Affichage du DataFrame
df_projobivoire

,Title,Type,DatePosted,DateClosing,Category,EmailURL,URL
0,Stagiaire cuisinier (iere),Stage,21 février 2024,10 décembre 2023,Restauration,https://projobivoire.com/jobs/stagiaire-cuisin...,https://projobivoire.com/page/0/
1,Directeur Financier -Banque – Sénégal,Emploi,21 février 2024,10 décembre 2023,Banque,https://projobivoire.com/jobs/directeur-financ...,https://projobivoire.com/page/0/
2,La PALMCI recrute Superviseur Sûreté – BAC,Emploi,21 février 2024,10 décembre 2023,Sécurité,https://projobivoire.com/jobs/la-palmci-recrut...,https://projobivoire.com/page/0/
3,Chef d’Agence Banque Cotonou,Emploi,21 février 2024,10 décembre 2023,Banque,https://projobivoire.com/jobs/chef-dagence-ban...,https://projobivoire.com/page/0/
4,UN(E) GÉRANT(E),Emploi,21 février 2024,10 décembre 2023,Commerce/Ventes,https://projobivoire.com/jobs/une-gerante/,https://projobivoire.com/page/0/
...,...,...,...,...,...,...,...
9277,Médecins sans Frontières- France recrute Assis...,Emploi,14 juin 2021,16 juillet 2021,Logistique,https://projobivoire.com/jobs/medecins-sans-fr...,https://projobivoire.com/page/545/
9278,OIM recrute Consultant(e) Junior,Emploi,14 juin 2021,16 juillet 2021,ONG,https://projobivoire.com/jobs/oim-recrute-cons...,https://projobivoire.com/page/545/
9279,ONG 2A recrute 4 Superviseurs de développement...,Emploi,14 juin 2021,16 juillet 2021,ONG,https://projobivoire.com/jobs/ong-2a-recrute-4...,https://projobivoire.com/page/545/
9280,Usine de Caoutchouc recrute plusieurs Profils,Emploi,14 juin 2021,16 juillet 2021,Usine,https://projobivoire.com/jobs/usine-de-caoutch...,https://projobivoire.com/page/545/


# Alerteemploi.net

In [33]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Liste des URLs des offres d'emploi
job_listing_urls = [
    "https://alerteemploi.net/toutes-les-offres/",
    # Ajoutez d'autres URLs au besoin
]

# Configurez Selenium pour s'exécuter en mode headless (sans ouvrir de fenêtre de navigateur)
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

# Liste pour stocker les données des offres d'emploi
job_data = []

# Boucle à travers chaque URL d'offre d'emploi
for url in job_listing_urls:
    # Envoyez une requête GET en utilisant Selenium
    driver.get(url)

    while True:
        try:
            # Cliquez sur le bouton "Load more listings"
            load_more_button = driver.find_element(By.CLASS_NAME, 'load_more_jobs')
            load_more_button.click()

            # Attendez le chargement des nouvelles offres d'emploi
            WebDriverWait(driver, 10).until(
                EC.invisibility_of_element_located((By.CLASS_NAME, 'loading_jobs'))
            )

            # Obtenez le code source de la page mis à jour
            page_source = driver.page_source

            # Utilisez BeautifulSoup pour analyser le HTML
            soup = BeautifulSoup(page_source, 'html.parser')

            # Trouvez toutes les offres d'emploi sur la page
            job_listings = soup.find_all('li', class_='job_listing')

            # Boucle à travers chaque offre d'emploi et extrayez les informations
            for listing in job_listings:
                job_title = listing.find('div', class_='position').find('h3').text.strip()
                company = listing.find('div', class_='company').find('strong').text.strip()
                location = listing.find('div', class_='location').text.strip()
                date_posted_element = listing.find('li', class_='date').find('time')
                date_posted = date_posted_element['datetime'].strip() if date_posted_element else None
                listing_url = listing.find('a')['href']

                # Stockez les données dans un dictionnaire
                job_entry = {
                    'Titre du poste': job_title,
                    'Entreprise': company,
                    'Lieu': location,
                    'Date de publication': date_posted,
                    'URL': listing_url,
                    'URL_offre': url
                    
                }

                job_data.append(job_entry)

        except Exception as e:
            job_entry = {
                    'Titre du poste': "",
                    'Entreprise': "",
                    'Lieu': "",
                    'Date de publication': "",
                    'URL': "",
                    'URL_offre': url
                    
                }
            # Sortez de la boucle si le bouton n'est pas trouvé ou s'il y a une exception
            break

# Fermez le pilote Selenium
driver.quit()

# Créez un DataFrame
df_alerteemploi = pd.DataFrame(job_data)

# Affichez le DataFrame
df_alerteemploi


,Titre du poste,Entreprise,Lieu,Date de publication,URL,URL_offre
0,Recrutement de livreurs à moto salaire 140.000fr,WENI,ABIDJAN,2023-01-10,https://alerteemploi.net/?post_type=job_listin...,https://alerteemploi.net/toutes-les-offres/
1,PROFIL DE POSTE – ASSISTANT(E) ADMINISTRATIF E...,TOP ASSISTANCE,ABIDJAN,2024-01-26,https://alerteemploi.net/p/profil-de-poste-ass...,https://alerteemploi.net/toutes-les-offres/
2,Assistant Projet chez Save The Children Intern...,SAVE THE CHILDREN,RCI,2024-01-21,https://alerteemploi.net/?post_type=job_listin...,https://alerteemploi.net/toutes-les-offres/
3,Emploi: Top postes disponibles pour novembre 2023,GROUP,RCI,2023-11-01,https://alerteemploi.net/?post_type=job_listin...,https://alerteemploi.net/toutes-les-offres/
4,Top recrutement chez Save the Chidren Internat...,SAVE THE CHILDREN,RCI,2023-10-29,https://alerteemploi.net/?post_type=job_listin...,https://alerteemploi.net/toutes-les-offres/
...,...,...,...,...,...,...
11995,Top stages à SGCI et UNICEF 2023,SGCI & UNICEF,ABIDJAN,2023-10-27,https://alerteemploi.net/?post_type=job_listin...,https://alerteemploi.net/toutes-les-offres/
11996,EMPLOI: recrutement de plusieurs postes TOP 20,GROUPS,RCI,2023-10-14,https://alerteemploi.net/?post_type=job_listin...,https://alerteemploi.net/toutes-les-offres/
11997,Nouvelle station recrute ses employés,STATION,RCI,2023-10-10,https://alerteemploi.net/?post_type=job_listin...,https://alerteemploi.net/toutes-les-offres/
11998,Best postes ouvert de la journée 27 septembre,GROUPS,RCI,2023-09-26,https://alerteemploi.net/?post_type=job_listin...,https://alerteemploi.net/toutes-les-offres/


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# Liste d'URLs des pages d'offres d'emploi
urls = list(df_alerteemploi["URL"])
# Initialiser une liste pour stocker les données
job_data = []

# Configurer Selenium pour s'exécuter en mode headless (sans ouvrir de fenêtre de navigateur)
chrome_options = Options()
chrome_options.add_argument("--headless")

# Boucle à travers chaque URL
for url in urls:
    try:
        # Initialiser le pilote Selenium
        driver = webdriver.Chrome(options=chrome_options)

        # Envoyer une requête GET en utilisant Selenium
        driver.get(url)

        # Attendre quelques secondes (ajustez selon les besoins)
        time.sleep(30)

        # Récupérer le code source de la page après l'exécution de JavaScript
        page_source = driver.page_source

        # Fermer le pilote Selenium
        driver.quit()

        # Utiliser BeautifulSoup pour analyser le HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extraire les détails de l'offre d'emploi
        job_title_element = soup.find('h1', class_='entry-title')
        job_title = job_title_element.text.strip() if job_title_element else None

        # Check if the initial element is found before attempting to find the nested element
        author_container = soup.find('div', class_='td-post-author-name')
        author_element = author_container.find('a') if author_container else None
        author = author_element.text.strip() if author_element else None

        date_posted_element = soup.find('time', class_='entry-date')
        date_posted = date_posted_element['datetime'].strip() if date_posted_element else None

        views_element = soup.find('div', class_='td-post-views')
        views = views_element.find('span', class_='td-nr-views-19100').text.strip() if views_element and views_element.find('span', class_='td-nr-views-19100') else None

        # Check if the element is found before accessing its properties
        image_url_element = soup.find('div', class_='td-post-featured-image')
        image_url = image_url_element.find('img')['src'] if image_url_element and image_url_element.find('img') else None

        # Ajouter les détails à la liste
        job_data.append({
            'Job Title': job_title,
            'Author': author,
            'Date Posted': date_posted,
            'Views': views,
            'Image URL': image_url,
            'URL': url  # Ajoutez l'URL de la page à la liste
        })
    except Exception as e:
        print(f"Une erreur s'est produite pour l'URL {url}: {str(e)}")
        job_data.append({
            'Job Title': "",
            'Author': "",
            'Date Posted': "",
            'Views': "",
            'Image URL': "",
            'URL': url  # Ajoutez l'URL de la page à la liste
        })

# Créer un DataFrame avec les données extraites

df = pd.DataFrame(job_data)

# Afficher le DataFrame
df


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# Liste d'URLs des pages d'offres d'emploi
urls = list(df_alerteemploi["URL"])
# Initialiser une liste pour stocker les données
job_data = []

# Configurer Selenium pour s'exécuter en mode headless (sans ouvrir de fenêtre de navigateur)
chrome_options = Options()
chrome_options.add_argument("--headless")

# Boucle à travers chaque URL
for url in urls:
    try:
        # Initialiser le pilote Selenium
        driver = webdriver.Chrome(options=chrome_options)

        # Envoyer une requête GET en utilisant Selenium
        driver.get(url)

        # Attendre quelques secondes (ajustez selon les besoins)
        time.sleep(30)

        # Attendre que la page soit entièrement chargée
        WebDriverWait(driver, 10).until(
            lambda x: x.execute_script("return document.readyState === 'complete'")
        )

        # Récupérer le code source de la page après l'exécution de JavaScript
        page_source = driver.page_source

        # Fermer le pilote Selenium
        driver.quit()

        # Utiliser BeautifulSoup pour analyser le HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extraire les détails de l'offre d'emploi
        job_title_element = soup.find('h1', class_='entry-title')
        job_title = job_title_element.text.strip() if job_title_element else None

        # Check if the initial element is found before attempting to find the nested element
        author_container = soup.find('div', class_='td-post-author-name')
        author_element = author_container.find('a') if author_container else None
        author = author_element.text.strip() if author_element else None

        date_posted_element = soup.find('time', class_='entry-date')
        date_posted = date_posted_element['datetime'].strip() if date_posted_element else None

        views_element = soup.find('div', class_='td-post-views')
        views = views_element.find('span', class_='td-nr-views-19100').text.strip() if views_element and views_element.find('span', class_='td-nr-views-19100') else None

        # Check if the element is found before accessing its properties
        image_url_element = soup.find('div', class_='td-post-featured-image')
        image_url = image_url_element.find('img')['src'] if image_url_element and image_url_element.find('img') else None

        # Ajouter les détails à la liste
        job_data.append({
            'Job Title': job_title,
            'Author': author,
            'Date Posted': date_posted,
            'Views': views,
            'Image URL': image_url,
            'URL': url  # Ajoutez l'URL de la page à la liste
        })
    except Exception as e:
        print(f"Une erreur s'est produite pour l'URL {url}: {str(e)}")
        job_data.append({
            'Job Title': "",
            'Author': "",
            'Date Posted': "",
            'Views': "",
            'Image URL': "",
            'URL': url  # Ajoutez l'URL de la page à la liste
        })

# Créer un DataFrame avec les données extraites
df = pd.DataFrame(job_data)

# Afficher le DataFrame
df


# jobcenter

In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Liste des URLs des pages d'offres d'emploi
urls = [
    "https://rmo-jobcenter.com/fr/nos-offres-emploi.html",
    # Ajoutez d'autres URLs au besoin
]

# Configurez Selenium pour s'exécuter en mode headless
chrome_options = Options()
chrome_options.add_argument("--headless")

# Liste pour stocker les données des offres d'emploi
all_job_data = []

# Base URL of the website
base_url = "https://rmo-jobcenter.com"

# Boucle à travers chaque URL
for url in urls:
    # Initialisez le pilote Selenium
    driver = webdriver.Chrome(options=chrome_options)

    # Chargez la page avec Selenium
    driver.get(url)

    # Récupérez le code source de la page après l'exécution du JavaScript
    page_source = driver.page_source

    # Fermez le pilote Selenium
    driver.quit()

    # Utilisez BeautifulSoup pour analyser le HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    # Trouvez la table contenant les offres d'emploi
    table = soup.find('table', class_='liste')

    # Liste pour stocker les données des offres d'emploi pour une URL spécifique
    job_data = []

    # Boucle à travers chaque ligne de la table (sauf la première qui contient les en-têtes)
    for row in table.find_all('tr')[1:]:
        # Extrayez les données de chaque colonne
        columns = row.find_all('td')
        date = columns[0].text.strip()
        filiale = columns[1].text.strip()
        fonction = columns[2].text.strip()
        secteur = columns[3].text.strip()
        reference = columns[4].text.strip()
        details_url = columns[5].find('a')['href']

        # Rendez l'URL absolue en la combinant avec l'URL de base.
        absolute_url = f"{base_url}/{details_url}"

        # Stockez les données dans un dictionnaire
        job_entry = {
            'Date': date,
            'Filiale': filiale,
            'Fonction': fonction,
            'Secteur': secteur,
            'Référence / Statut': reference,
            'Détails URL': absolute_url,
            "URL": url
        }

        job_data.append(job_entry)

    # Ajoutez les données de cette URL à la liste globale
    all_job_data.extend(job_data)

# Créez un DataFrame avec toutes les données extraites
df_jobcenter = pd.DataFrame(all_job_data)

# Affichez le DataFrame
df_jobcenter


In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Liste des URLs des pages d'offres d'emploi
urls = list(df_jobcenter['Détails URL'])

# Configurez Selenium pour s'exécuter en mode headless
chrome_options = Options()
chrome_options.add_argument("--headless")

# Liste pour stocker les données des offres d'emploi
all_job_data = []

# Boucle à travers chaque URL
for url in urls:
    # Initialisez le pilote Selenium
    driver = webdriver.Chrome(options=chrome_options)

    # Chargez la page avec Selenium
    driver.get(url)

    # Récupérez le code source de la page après l'exécution du JavaScript
    page_source = driver.page_source

    # Fermez le pilote Selenium
    driver.quit()

    # Utilisez BeautifulSoup pour analyser le HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    # Trouvez la div contenant les informations détaillées
    details_div = soup.find('div', {'id': 'content_articles'})

    # Extract details from the div
    job_title = details_div.find('div', {'id': 'h2_imprime'}).text.strip()
    job_description = details_div.find('div', {'class': 'text-content'}).text.strip()

    # Additional details can be extracted similarly

    # Stockez les données dans un dictionnaire
    job_entry = {
        'Job Title': job_title,
        'Job Description': job_description,
        'URL': url
        # Add more details as needed
    }

    # Ajoutez les données de cette URL à la liste globale
    all_job_data.append(job_entry)

# Créez un DataFrame avec toutes les données extraites
df_details = pd.DataFrame(all_job_data)

# Affichez le DataFrame
df_details


# yop.l-frii

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# List of URLs
urls = [
    "https://yop.l-frii.com/offres-demplois/{}/".format(category) for category in range(3276)
    # Add more URLs as needed
]

job_data = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    job_articles = soup.find_all('article', class_='type-emploi')

    for article in job_articles:
        # Check if the 'h2' element with class 'elementor-heading-title' is found
        job_title_element = article.find('h2', class_='elementor-heading-title')
        job_title = job_title_element.text.strip() if job_title_element else None

        job_link = article.find('a', href=True)['href']
        
        # Check if the image element is found
        job_image_element = article.find('img', class_='attachment-large')
        job_image = job_image_element['src'] if job_image_element else None

        job_data.append({
            "Job Title": job_title,
            "Job Link": job_link,
            "Job Image": job_image,
            "Source URL": url  # Include the source URL in the DataFrame
        })

df_yop_l_frii = pd.DataFrame(job_data)
df_yop_l_frii


# unjobnet.org

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fonction pour extraire les informations d'une annonce
def extract_job_info(job):
    title = job.find('a', class_='py-2 h6 fw-bold').text.strip()
    organization = job.find('a', class_='link-dark').text.strip()
    location = job.find_all('a', class_='text-darkx')[1].text.strip()
    country = job.find_all('a', class_='text-darkx')[2].text.strip()
    category = job.find('span', class_='ms-1 me-3').text.strip()
    contract_type = job.find_all('a', class_='ms-1 me-3')[1].text.strip()
    posted = job.find('span', class_='text-muted').text.strip()
    close_date = job.find('span', class_='ms-1').text.strip()
    
    return {
        'Title': title,
        'Organization': organization,
        'Location': location,
        'Country': country,
        'Category': category,
        'Contract Type': contract_type,
        'Posted': posted,
        'Close Date': close_date
    }

# URL de la page contenant les annonces
url = "https://www.unjobnet.org/organizations/WHO/jobs"

# Faire une requête GET à l'URL
response = requests.get(url)

# Analyser le contenu HTML de la page
soup = BeautifulSoup(response.content, 'html.parser')

# Trouver toutes les annonces d'emploi
jobs = soup.find_all('div', class_='row p-3 border-top')

# Extraire les informations de chaque annonce
job_info_list = [extract_job_info(job) for job in jobs]

# Créer un DataFrame à partir de la liste d'informations sur les emplois
df = pd.DataFrame(job_info_list)

# Afficher le DataFrame
df
